In [1]:
#get json ds

##link to egfr_10_full_ft_pd_lines.json
#https://drive.google.com/file/d/1r_hC1sa1H2Ht3ZGlp-rkWbcN1a-CIr8I/view?usp=sharing

#!gdown --id "1r_hC1sa1H2Ht3ZGlp-rkWbcN1a-CIr8I"
#!gdown --id "1pWwaJt8WSjBfZK5r-rxbO-svE43spPek"
#!gdown --id "1MVoKgsUFx5mdGNtZ7uSsCtygSOsrzu8u"
!gdown --id "1W2gaP_8LdOjtkBKTxWItwwB2iNLxo2Fx"
path2data_full = '/content/hiv1_hcv.csv'

Downloading...
From: https://drive.google.com/uc?id=1W2gaP_8LdOjtkBKTxWItwwB2iNLxo2Fx
To: /content/hiv1_hcv.csv
5.44GB [00:51, 105MB/s] 


In [ ]:
'''
use subset of the original hiv1_hcv if you don't have 20Gb RAM!
'''

import pandas as pd
subset = pd.read_csv('/content/hiv1_hcv.csv').sample(frac=0.50)
subset.to_csv('/content/subset_hiv1_hcv.csv', index=False)
del subset
path2data_subset = '/content/subset_hiv1_hcv.csv'

In [2]:
!pip install ray[default]
!pip install tensorboard_logger
!pip install tensorboardX

     |████████████████████████████████| 49.7MB 88kB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 133kB 71.0MB/s 
     |████████████████████████████████| 1.0MB 37.5MB/s 
     |████████████████████████████████| 1.3MB 54.3MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 3.1MB 52.9MB/s 
     |████████████████████████████████| 204kB 47.8MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 296kB 58.4MB/s 
     |████████████████████████████████| 143kB 62.2MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=53a60fc545d25044a3e9cd78f6c2f926f538355b1229165483715496b8f4906f
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
  Found existing installation: protobuf 3.12.4

     |████████████████████████████████| 122kB 3.9MB/s 


In [3]:
import os
#if not os.path.isdir('checkpoint'):
    #!mkdir 'checkpoint'

# checkpoint_dir = 'checkpoint/'

import copy
import warnings
import json
from filelock import FileLock

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import dataloader
import torch.optim as optim
from sklearn.metrics import precision_recall_curve
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial


from nets import UnitedNet
from dataset import ANYDataset, train_validation_split
from metrics import *
import utils

warnings.filterwarnings('ignore')

In [4]:
CONFIG_PARAM = ['lr', 'l2reg', 'dropout']
METRIC_PARAM=["train_loss", "train_sensitivity", "train_specificity", "train_accuracy", "train_mcc", "train_auc", 
              "val_loss", "val_sensitivity", "val_specificity", "val_accuracy", "val_mcc", "val_auc"]
              
DATASET = '/content/hiv1_hcv.csv'
#DATASET = '/content/subset_hiv1_hcv.csv'
#train_data, val_data = train_validation_split(DATASET)
#del train_data, val_data

NEPOCH = 500
BATCHSIZE = 128
HASHCODE = 'TEST'
USE_MAT = True
USE_MORD = True

METRICS_DICT = {
    'sensitivity': sensitivity,
    'specificity': specificity,
    'accuracy': accuracy,
    'mcc': mcc,
    'auc': auc,
}

if torch.cuda.is_available():
    TRAIN_DEVICE = 'cuda'
    VAL_DEVICE = 'cuda'
else:
    TRAIN_DEVICE = 'cpu'
    VAL_DEVICE = 'cpu'

In [5]:
config = {
    'lr': tune.loguniform(5e-6, 1e-4),
    'l2reg': tune.loguniform(5e-6, 1),
    'dropout': tune.uniform(0.0, 0.7),
}

In [6]:
def UnitedNet_wraper(config, dense_dim, smiles_len, use_mord=True, use_mat=True, infer=False, dir_path=None):
    return UnitedNet(dense_dim, smiles_len, use_mord=use_mord, use_mat=use_mat, infer=infer, dir_path=dir_path, 
                 vis_thresh=0.2, dropout=config['dropout'])

*****SINGLE RUN HT*****

In [7]:
def run_epoch(net, opt, criterion, train_loader, val_loader):

    metrics = dict((metric, []) for metric in METRIC_PARAM)

    # Train step
    train_losses = []
    train_outputs = []
    train_labels = []
    net.train()
    for i, (mord_ft, non_mord_ft, label) in enumerate(train_loader):
        mord_ft = mord_ft.float().to(TRAIN_DEVICE)
        non_mord_ft = non_mord_ft.view((-1, 1, int(non_mord_ft.shape[1]/42), 42)).float().to(TRAIN_DEVICE)
        mat_ft = non_mord_ft.squeeze(1).float().to(TRAIN_DEVICE)
        label = label.float().to(TRAIN_DEVICE)

        # Forward
        opt.zero_grad()

        outputs = net(non_mord_ft, mord_ft, mat_ft)
        outputs = torch.squeeze(outputs)
        
        loss = criterion(outputs, label)
        train_losses.append(float(loss.item()))
        train_outputs.extend(outputs)
        train_labels.extend(label)

        # Parameters update
        loss.backward()
        opt.step()
    
    # Validation step
    val_losses = []
    val_outputs = []
    val_labels = []
    net.eval()
    for i, (mord_ft, non_mord_ft, label) in enumerate(val_loader):
        mord_ft = mord_ft.float().to(VAL_DEVICE)
        non_mord_ft = non_mord_ft.view((-1, 1, int(non_mord_ft.shape[1]/42), 42)).float().to(VAL_DEVICE)
        mat_ft = non_mord_ft.squeeze(1).float().to(VAL_DEVICE)
        label = label.float().to(VAL_DEVICE)

        with torch.no_grad():
            outputs = net(non_mord_ft, mord_ft, mat_ft)
            outputs = torch.squeeze(outputs)
            
            loss = criterion(outputs, label)
            val_losses.append(float(loss.item()))
            val_outputs.extend(outputs)
            val_labels.extend(label)

    # Calculate metrics
    train_outputs = torch.stack(train_outputs)
    val_outputs = torch.stack(val_outputs)
    train_labels = torch.stack(train_labels)
    val_labels = torch.stack(val_labels)

    metrics['train_loss'] = sum(train_losses) / len(train_losses)
    metrics['val_loss'] = sum(val_losses) / len(val_losses)
    for key in METRICS_DICT.keys():
        metrics['train_' + key] = METRICS_DICT[key](train_labels, train_outputs)
        metrics['val_' + key] = METRICS_DICT[key](val_labels, val_outputs)

    for metric, measurements in metrics.items():
        metrics[metric] = np.mean(metrics[metric])

    return metrics

In [8]:
def single_run_tune(config, nepoch, opt_type='adam', dense_dim=None, train_loader=None, val_loader=None):

    def make_loaders(dataset, batch_size):
        train_data, val_data = train_validation_split(dataset)
        train_dataset = ANYDataset(train_data)
        del train_data
        train_size = len(train_dataset)
        dense_dim = train_dataset.get_dim('mord')
        train_loader = dataloader.DataLoader(dataset=train_dataset, batch_size=batch_size, 
                                                collate_fn=utils.custom_collate, shuffle=True)
        del train_dataset

        val_dataset = ANYDataset(val_data)
        del val_data
        val_size = len(val_dataset)
        val_loader = dataloader.DataLoader(dataset=val_dataset,batch_size=batch_size,
                                            collate_fn=utils.custom_collate, shuffle=False)
        del val_dataset
        print('Dataloaders created: train {}, val {}'.format(train_size, val_size))
        return train_loader, val_loader, dense_dim

    lock = FileLock(DATASET + ".lock")
    with lock:
        train_loader, val_loader, dense_dim = make_loaders(DATASET, BATCHSIZE)
    
    mord_ft, non_mord_ft, label = next(iter(train_loader))
    smiles_len = int(non_mord_ft.shape[1]/42)

    criterion = nn.BCELoss()

    net = UnitedNet_wraper(config, dense_dim, smiles_len=smiles_len, 
                           use_mord=USE_MORD, use_mat=USE_MAT, infer=False, dir_path=None).to(TRAIN_DEVICE)
 
    if opt_type == 'sgd':
        opt = optim.SGD(net.parameters(), lr=config['lr'], momentum=0.99)
    elif opt_type == 'adam':
        opt = optim.Adam(net.parameters(), lr=config['lr'], weight_decay=config['l2reg'])

    '''
    if checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        try:
            model_state, optimizer_state = torch.load(path)
            net.load_state_dict(model_state)
            opt.load_state_dict(optimizer_state)
        except:
            print('Cant load the model')
    '''

    '''
    start_epoch = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            start = state["step"] + 1
    '''

    for epoch in range(nepoch):
        metrics = run_epoch(net, opt, criterion, train_loader, val_loader)
        
        '''
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), opt.state_dict()), path)
        '''
        '''
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            with open(path, "w") as f:
                f.write(json.dumps({"step": epoch}))
        '''
        tune.report(
            train_loss=metrics['train_loss'], 
            train_sensitivity=metrics['train_sensitivity'],
            train_specificity=metrics['train_specificity'],
            train_accuracy=metrics['train_accuracy'],
            train_mcc=metrics['train_mcc'],
            train_auc=metrics['train_auc'],
            val_loss=metrics['val_loss'],
            val_sensitivity=metrics['val_sensitivity'],
            val_specificity=metrics['val_specificity'],
            val_accuracy=metrics['val_accuracy'],
            val_mcc=metrics['val_mcc'],
            val_auc=metrics['val_auc']
        )


In [11]:
def main(config, num_samples=4, max_concurrent=4, grace_period=4, reduction_factor=4, max_num_epochs=30,
         train_loader=None, val_loader=None, dense_dim=None):
    
    target_metric = 'val_mcc'
    target_direction = 'max'
    gpus_per_trial = torch.cuda.device_count() / max_concurrent
    cpu_per_trial = os.cpu_count() / max_concurrent
    print('gpus_per_trial:', gpus_per_trial, "cpu_per_trial:", cpu_per_trial)

    scheduler = ASHAScheduler(
        metric=target_metric,
        mode=target_direction,
        max_t=max_num_epochs,
        grace_period=grace_period,
        reduction_factor=reduction_factor)

    reporter = CLIReporter(
        parameter_columns=CONFIG_PARAM,
        metric_columns=["training_iteration", METRIC_PARAM[0]] + METRIC_PARAM[6:])

    result = tune.run(
        partial(single_run_tune,
                nepoch=max_num_epochs, 
                opt_type='adam',
                dense_dim=dense_dim,
                train_loader=train_loader,
                val_loader=val_loader),
        name="asha_ht",
        resources_per_trial={"cpu": cpu_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        verbose=2,
        stop={"training_iteration": max_num_epochs})

    best_trial = result.get_best_trial("val_loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["val_loss"]))

    return result



In [12]:
result = main(config, num_samples=12, max_concurrent=1, grace_period=32, reduction_factor=2, max_num_epochs=128)
#              train_loader=train_loader, val_loader=val_loader, dense_dim=dense_dim)

2021-05-27 09:18:08,000	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-05-27 09:18:08,002	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


gpus_per_trial: 1.0 cpu_per_trial: 4.0
== Status ==
Memory usage on this node: 1.5/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 128.000: None | Iter 64.000: None | Iter 32.000: None
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (11 PENDING, 1 RUNNING)


== Status ==
Memory usage on this node: 2.0/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 128.000: None | Iter 64.000: None | Iter 32.000: None
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (11 PENDING, 1 RUNNING)


(pid=291) Train
(pid=291) (87151, 865)
(pid=291) Val
(pid=291) (21788, 865)
(pid=291) Dataloaders created: train 87151, val 21788
Trial DEFAULT_7330b_00000 reported train_loss=0.5254245751842354,train_sensitivity=0.8370880177830975

2021-05-27 15:05:10,350	WARNING worker.py:1115 -- The actor or task with ID ffffffffffffffff49ff7e41818226ea6acf375701000000 cannot be scheduled right now. It requires {CPU_group_beb858145da419d5e3dd1fecdeab5cf9: 4.000000}, {CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 4.000000}, {GPU_group_beb858145da419d5e3dd1fecdeab5cf9: 1.000000}, {GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 1.000000} for placement, but this node only has remaining {0.000000/4.000000 CPU, 15.005200 GiB/15.005200 GiB memory, 0.000000/1.000000 GPU, 7.502600 GiB/7.502600 GiB object_store_memory, 0.000000/1.000000 GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/4.000000 CPU_group_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/1.000000 GPU_group_beb858145da419d5e3dd1fecdeab5cf9, 1.000000/1.000000 accelerator_type:P100, 1.000000/1.000000 node:172.28.0.2, 0.000000/4.000000 CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9}
. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all c

(pid=289) Closing files ...
(pid=437) Dataloaders created: train 87151, val 21788
Trial DEFAULT_7330b_00003 reported train_loss=0.4471907663362842,train_sensitivity=0.7969050570683538,train_specificity=0.6049436933404435,train_accuracy=0.6564698052804901,train_mcc=0.35615410422802557,train_auc=0.7978188397173009,val_loss=0.38091027928374666,val_sensitivity=0.8906143344709898,val_specificity=0.5670517327975891,val_accuracy=0.6540756379658528,val_mcc=0.4084471237335921,val_auc=0.8649994782408208 with parameters={'lr': 9.912558035577727e-05, 'l2reg': 0.038850812933101754, 'dropout': 0.33213714800837324}.
== Status ==
Memory usage on this node: 22.5/25.5 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 128.000: 0.4438618982496668 | Iter 64.000: 0.41201933938160673 | Iter 32.000: 0.465844476815517
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (8 PENDIN

2021-05-27 17:00:10,608	WARNING worker.py:1115 -- The actor or task with ID ffffffffffffffffa949be09653e9aa125e3c02901000000 cannot be scheduled right now. It requires {CPU_group_beb858145da419d5e3dd1fecdeab5cf9: 4.000000}, {CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 4.000000}, {GPU_group_beb858145da419d5e3dd1fecdeab5cf9: 1.000000}, {GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 1.000000} for placement, but this node only has remaining {0.000000/4.000000 CPU, 15.005200 GiB/15.005200 GiB memory, 0.000000/1.000000 GPU, 7.502600 GiB/7.502600 GiB object_store_memory, 0.000000/1.000000 GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/4.000000 CPU_group_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/1.000000 GPU_group_beb858145da419d5e3dd1fecdeab5cf9, 1.000000/1.000000 accelerator_type:P100, 1.000000/1.000000 node:172.28.0.2, 0.000000/4.000000 CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9}
. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all c

...

(pid=610) Closing files ...
(pid=639) Dataloaders created: train 87151, val 21788
Trial DEFAULT_7330b_00010 reported train_loss=0.4850788548829447,train_sensitivity=0.7819005685461463,train_specificity=0.5711283289940086,train_accuracy=0.6277036408073344,train_mcc=0.313230310139559,train_auc=0.7605410472589851,val_loss=0.4580586821015118,val_sensitivity=0.8402730375426621,val_specificity=0.5379834254143646,val_accuracy=0.6192858454194969,val_mcc=0.33821082374136874,val_auc=0.8018229912164466 with parameters={'lr': 0.0008208668101534388, 'l2reg': 0.09786702163025562, 'dropout': 0.5224014127177476}.
== Status ==
Memory usage on this node: 22.6/25.5 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 128.000: 0.6210120726552467 | Iter 64.000: 0.5860996542383998 | Iter 32.000: 0.51715735172151
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (1 PENDING, 1 R

2021-05-28 02:41:51,761	WARNING worker.py:1115 -- The actor or task with ID ffffffffffffffff2ae5badda74cd0f0b701731201000000 cannot be scheduled right now. It requires {GPU_group_beb858145da419d5e3dd1fecdeab5cf9: 1.000000}, {CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 4.000000}, {GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9: 1.000000}, {CPU_group_beb858145da419d5e3dd1fecdeab5cf9: 4.000000} for placement, but this node only has remaining {0.000000/4.000000 CPU, 15.005200 GiB/15.005200 GiB memory, 0.000000/1.000000 GPU, 7.502600 GiB/7.502600 GiB object_store_memory, 0.000000/1.000000 GPU_group_0_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/4.000000 CPU_group_beb858145da419d5e3dd1fecdeab5cf9, 0.000000/1.000000 GPU_group_beb858145da419d5e3dd1fecdeab5cf9, 1.000000/1.000000 accelerator_type:P100, 1.000000/1.000000 node:172.28.0.2, 0.000000/4.000000 CPU_group_0_beb858145da419d5e3dd1fecdeab5cf9}
. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all c

(pid=639) Closing files ...
(pid=668) Dataloaders created: train 87151, val 21788
Trial DEFAULT_7330b_00011 reported train_loss=0.4748916294693772,train_sensitivity=0.7815158380712179,train_specificity=0.5953448978951661,train_accuracy=0.6453167490906587,train_mcc=0.33402437174029165,train_auc=0.7740933076725843,val_loss=0.4309242894077859,val_sensitivity=0.841296928327645,val_specificity=0.5835635359116023,val_accuracy=0.6528823205434184,val_mcc=0.377492936485106,val_auc=0.8240559372980459 with parameters={'lr': 0.0005627490213207546, 'l2reg': 0.03477010620362768, 'dropout': 0.6188752274592156}.
== Status ==
Memory usage on this node: 22.5/25.5 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 128.000: 0.6210120726552467 | Iter 64.000: 0.5860996542383998 | Iter 32.000: 0.5115895163470395
Resources requested: 4.0/4 CPUs, 1.0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (1 RUNNING, 11

2021-05-28 03:12:48,038	INFO tune.py:549 -- Total run time: 64480.04 seconds (64479.85 seconds for the tuning loop).


Trial DEFAULT_7330b_00011 reported train_loss=0.45357066375393523,train_sensitivity=0.7874150386867866,train_specificity=0.6331440760375169,train_accuracy=0.6745533614072128,train_mcc=0.37303683867116033,train_auc=0.7973356309341303,val_loss=0.43228638207006176,val_sensitivity=0.8098976109215017,val_specificity=0.669952285283777,val_accuracy=0.7075913346796402,val_mcc=0.426977420200046,val_auc=0.8311136140790555 with parameters={'lr': 0.0005627490213207546, 'l2reg': 0.03477010620362768, 'dropout': 0.6188752274592156}. This trial completed.
== Status ==
Memory usage on this node: 22.2/25.5 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 128.000: 0.6210120726552467 | Iter 64.000: 0.5860996542383998 | Iter 32.000: 0.5005990377870526
Resources requested: 0/4 CPUs, 0/1 GPUs, 0.0/15.01 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/asha_ht
Number of trials: 12/12 (12 TERMINATED)


== Status ==
Memory usage on this node: 22.2/25.5 GiB
Usin

In [13]:
import pandas as pd

future_df = {}
for k, v in result.results.items():
    observ = {}
    for conf in CONFIG_PARAM:
        observ[conf] = result.results[k]['config'][conf]
    for metric in METRIC_PARAM + ['training_iteration']:
        observ[metric] = result.results[k][metric]
    future_df[k] = observ

df = pd.DataFrame.from_dict(future_df, orient='index')
df = df.sort_values('val_mcc', ascending=False)
df

,lr,l2reg,dropout,train_loss,train_sensitivity,train_specificity,train_accuracy,train_mcc,train_auc,val_loss,val_sensitivity,val_specificity,val_accuracy,val_mcc,val_auc,training_iteration
7330b_00009,0.000016,0.000156,0.528336,0.145718,0.939683,0.909643,0.917706,0.807952,0.981876,0.305243,0.787372,0.899360,0.869240,0.674323,0.918431,128
7330b_00006,0.000013,0.076754,0.421737,0.172806,0.967512,0.923633,0.935411,0.849718,0.988161,0.309982,0.779522,0.891135,0.861116,0.655875,0.900890,128
7330b_00002,0.000034,0.001854,0.637467,0.146840,0.941735,0.913172,0.920839,0.814655,0.981777,0.287889,0.825256,0.853277,0.845741,0.640252,0.916721,128
7330b_00003,0.000099,0.038851,0.332137,0.308225,0.851964,0.812918,0.823398,0.611801,0.913775,0.311829,0.813823,0.846434,0.837663,0.622085,0.907520,128
7330b_00005,0.000005,0.000025,0.376992,0.231702,0.881973,0.852960,0.860747,0.685427,0.948009,0.285483,0.816724,0.842918,0.835873,0.619940,0.913454,128
7330b_00008,0.000090,0.100050,0.054485,0.340562,0.859573,0.791540,0.809801,0.593476,0.910451,0.354040,0.816894,0.791939,0.798651,0.558604,0.889324,128
7330b_00001,0.000236,0.056993,0.591475,0.432703,0.809088,0.649393,0.692258,0.406904,0.823129,0.411353,0.820307,0.678240,0.716449,0.443862,0.850603,128
7330b_00011,0.000563,0.034770,0.618875,0.453571,0.787415,0.633144,0.674553,0.373037,0.797336,0.432286,0.809898,0.669952,0.707591,0.426977,0.831114,32
7330b_00010,0.000821,0.097867,0.522401,0.543322,0.889369,0.186988,0.375521,0.090831,0.658272,0.543180,0.746246,0.431630,0.516247,0.162187,0.667802,32
7330b_00000,0.000061,0.324821,0.522496,0.549996,0.987347,0.018006,0.278195,0.018580,0.651215,0.549517,0.991126,0.010422,0.274188,0.006898,0.654939,128


In [14]:
file_name = 'asha_Data1_12_base_mcc'
df.to_csv(file_name + '.csv')

In [15]:
df_small = df[CONFIG_PARAM + ['training_iteration', 'train_loss', "val_loss", "val_sensitivity", "val_specificity", "val_accuracy", "val_mcc", "val_auc"]]
df_small.to_csv(file_name + '_small.csv')
df_small.head(3)

,lr,l2reg,dropout,training_iteration,train_loss,val_loss,val_sensitivity,val_specificity,val_accuracy,val_mcc,val_auc
7330b_00009,0.000016,0.000156,0.528336,128,0.145718,0.305243,0.787372,0.899360,0.869240,0.674323,0.918431
7330b_00006,0.000013,0.076754,0.421737,128,0.172806,0.309982,0.779522,0.891135,0.861116,0.655875,0.900890
7330b_00002,0.000034,0.001854,0.637467,128,0.146840,0.287889,0.825256,0.853277,0.845741,0.640252,0.916721
